<a href="https://colab.research.google.com/github/KusaLLL/Test-data-set-python/blob/main/testdataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

import pandas as pd
path = '/content/drive/MyDrive/ml last project/Tweets.csv'
df = pd.read_csv(path)



df.describe







Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<bound method NDFrame.describe of                  tweet_id airline_sentiment  airline_sentiment_confidence  \
0      570306133677760513           neutral                        1.0000   
1      570301130888122368          positive                        0.3486   
2      570301083672813571           neutral                        0.6837   
3      570301031407624196          negative                        1.0000   
4      570300817074462722          negative                        1.0000   
...                   ...               ...                           ...   
14635  569587686496825344          positive                        0.3487   
14636  569587371693355008          negative                        1.0000   
14637  569587242672398336           neutral                        1.0000   
14638  569587188687634433          negative                        1.0000   
14639  569587140490866689           neutral                        0.6771   

               negativereason  negativere

In [ ]:
df = df[["airline_sentiment", "text"]]

import nltk
import string
import re
from nltk.stem.porter import PorterStemmer

nltk.download('stopwords')
from nltk.corpus import stopwords

nltk.download('punkt')
ps = PorterStemmer()

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http.?://[^\s]+[\s]?', '', text)
    text = nltk.word_tokenize(text)
    y = []
    for i in text:
        if i not in stopwords.words('english'):
            y.append(i)
    text = y[:]
    y.clear()
    for i in text:
        y.append(ps.stem(i))
    return " ".join(y)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df['text_cleaned'] = df['text'].apply(clean_text)


<ipython-input-18-32b16c72a5ef>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text_cleaned'] = df['text'].apply(clean_text)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# create a TfidfVectorizer object with max_features=3000
vectorizer = TfidfVectorizer(max_features=3000)

# use the fit_transform() method to create a TF-IDF matrix from the 'text_cleaned' column of the DataFrame
tfidf_matrix = vectorizer.fit_transform(df['text_cleaned'])

# print the shape of the matrix
print(tfidf_matrix.shape)



(14640, 3000)


In [ ]:
# use the TfidfVectorizer object to transform the 'text_cleaned' column of the DataFrame
tfidf_matrix = vectorizer.transform(df['text_cleaned'])

# convert the sparse matrix to a dense array
X = tfidf_matrix.toarray()

# print the shape of the array
print(X.shape)


(14640, 3000)


In [ ]:
import numpy as np

# convert the 'airline_sentiment' column of the DataFrame to a NumPy array
Y = np.array(df['airline_sentiment'])

# print the shape of the array
print(Y.shape)


(14640,)


In [ ]:
from sklearn.model_selection import train_test_split

# split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=2)

# print the shapes of the resulting arrays
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)


X_train shape: (11712, 3000)
X_test shape: (2928, 3000)
Y_train shape: (11712,)
Y_test shape: (2928,)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# create a Multinomial Naïve Bayes classifier
clf = MultinomialNB()

# fit the classifier to the training data
clf.fit(X_train, Y_train)

# predict the sentiment of tweets in the test dataset
Y_pred = clf.predict(X_test)

# calculate the accuracy of the model in predicting sentiment labels for the test dataset
accuracy = accuracy_score(Y_test, Y_pred)

# print the accuracy score
print("Accuracy:", accuracy)


Accuracy: 0.7219945355191257


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# create a Random Forest classifier
rfc = RandomForestClassifier()

# fit the classifier to the training data
rfc.fit(X_train, Y_train)

# predict the sentiment of tweets in the test dataset
Y_pred = rfc.predict(X_test)

# calculate the accuracy of the model in predicting sentiment labels for the test dataset
accuracy = accuracy_score(Y_test, Y_pred)

# print the accuracy score
print("Accuracy:", accuracy)


Accuracy: 0.7517076502732241


In [ ]:
# Apply the clean_text function to the first text in the 'text' column
preprocessed_text = clean_text(df['text'][0])

# Print the result
print(preprocessed_text)


@ virginamerica @ dhepburn said .


In [ ]:
num_negative = (df['airline_sentiment'] == 'negative').sum()
print('Number of instances with negative sentiment:', num_negative)


Number of instances with negative sentiment: 9178


In [ ]:
num_unique_neutral = df[df['airline_sentiment'] == 'neutral']['text'].nunique()
print('Number of unique tweets with neutral sentiment:', num_unique_neutral)


Number of unique tweets with neutral sentiment: 3067
